In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_hours = (
    pd.read_csv("hours_exploded.csv")
    .astype({"Datum": "datetime64[s]"})
    .query("Datum > '2025-09-01'")  # filter for Kita year 2025/2026
)
df_names = pd.read_csv("names.csv")

# Add family column
df_names = df_names.assign(
    Familie=np.where(
        (df_names["Nachname Mutter"] == df_names["Nachname Vater"])
        | df_names["Nachname Vater"].isna(),
        df_names["Nachname Mutter"],
        df_names["Nachname Mutter"] + " + " + df_names["Nachname Vater"],
    )
    # IMPORTANT: This ignore the case of single-parent fathers
).assign(alleinerziehend=lambda x: x["Nachname Vater"].isna())

In [3]:
df_hours

,Datum,Kommentare,Stunden,wer?,Kategorie,wer?_id,wer?_type,wer?_displayName
0,2025-09-23,Erstellung Stundenaufzeichnungstabelle,1.0,NaN,Digitalamt,b.zorn,0.0,Benedikt Zorn
1,2025-09-23,Vorbereitung Bautag,0.5,NaN,Bautag,m.hirsch,0.0,Maren Hirsch
2,2025-09-26,Ausarbeitung Stundenprotokoll,2.5,NaN,Verwaltung,m.hirsch,0.0,Maren Hirsch
3,2025-09-27,Bautag,6.0,NaN,Bautag,m.hirsch,0.0,Maren Hirsch
4,2025-09-29,Vollversammlung,2.0,NaN,Vollversammlung / 2 h,h.lauppe,0.0,Heiner Lauppe
...,...,...,...,...,...,...,...,...
265,2025-11-15,NaN,4.0,NaN,Bautag,r.bessai,0.0,Rosa Bessai
266,2025-11-13,Dokumente erstellen …,5.0,NaN,Vorstand,r.bessai,0.0,Rosa Bessai
267,2025-11-15,NaN,3.0,NaN,Bautag,t.schaffitzel,0.0,Timo Schaffitzel
268,2025-11-15,NaN,4.0,NaN,Bautag,k.seuffert,0.0,Katharina Seuffert


In [4]:
hours_dict = df_hours.groupby("wer?_id")["Stunden"].sum().to_dict()

In [5]:
df_names.head()

,Vorname Kind,Nachname Kind,Geburtsdatum,Gruppe,Straße,PLZ,Ort,Nextcloudaccount Mutter,Nextcloudaccount Vater,Vorname Mutter,Nachname Mutter,Email Mutter,Telefon Mutter,Vorname Vater,Nachname Vater,Email Vater,Telefon Vater,Familie,alleinerziehend
0,Alma,Baldas,03.06.2022,Kita,Etllinger Str. 47,76137,Karlsruhe,m.baldas,r.baldas,Mirjam,Baldas,Mirjam.baldas@gmx.de,0176 82469383,Ruben,Baldas,Ruben.baldas@mail.de,0176 31418126,Baldas,False
1,Arto,Bube,14.02.2019,Kita,Treischkestr. 2,76137,Karlsruhe,k.lautenbach,p.bube,Katrin,Lautenbach,mail@katrinlautenbach.de,0176 64015453,Peter,Bube,peter.bube@gmx.de,0176 22348065,Lautenbach + Bube,False
2,August,Maercks,20.01.2020,Kita,Hirschstr. 45,76133,Karlsruhe,a.maercks,m.maercks,Annika,Maercks,annika@maercks.eu,0157 88213981,Marius,Maercks,marius@maercks.eu,0157 50985393,Maercks,False
3,Bea,Hu,14.06.2023,Krabbel,"Kaiserstraße 179,",76133,Karlsruhe,q.dong,y.hu,Qianzhi,Dong,dongqianzhi@gmail.com,017640250088,Ying,Hu,jackieyh@gmail.com,017624020568,Dong + Hu,False
4,Bela,Weinfurtner,25.01.2021,Kita,Geranienstraße 13,76185,Karlsruhe,c.schermann,t.weinfurtner,Corina,Weinfurtner,c.schermann@gmx.de,01514 1825797,Thomas,Weinfurtner,weinfurtner.thomas@gmail.com,0160 8350358,Weinfurtner,False


In [6]:
df_names.query("alleinerziehend")

,Vorname Kind,Nachname Kind,Geburtsdatum,Gruppe,Straße,PLZ,Ort,Nextcloudaccount Mutter,Nextcloudaccount Vater,Vorname Mutter,Nachname Mutter,Email Mutter,Telefon Mutter,Vorname Vater,Nachname Vater,Email Vater,Telefon Vater,Familie,alleinerziehend
25,Lotta,Langer,02.03.2020,Kita,Nelkenstr. 31,76135,Karlsruhe,l.hofmann,m.langer,Lena,Hofmann,lvm.hofmann@gmail.com,0157 72586595,NaN,NaN,NaN,NaN,Hofmann,True


In [7]:
# Time Targets: 
# 1 Kind = 102
# 2 Kinder = 132
# 3 Kinder = ?
# Alleinerziehende:
# 1 Kind = 50
# 2 Kinder = 60

target_hours_dict = {
    (False, 1): 102,
    (False, 2): 132,
    (False, 3): 132, # to be determined
    (True, 1): 50,
    (True, 2): 60,
}
children_count = df_names.groupby("Familie")["Vorname Kind"].count().sort_values().to_dict()

In [30]:
family_hours = (
        df_names[
            [
                "Familie",
                "alleinerziehend",
                "Nextcloudaccount Mutter",
                "Nextcloudaccount Vater",
            ]
        ].drop_duplicates(subset=["Familie"])
        .assign(stunden1=lambda x: x["Nextcloudaccount Mutter"].map(hours_dict).fillna(0))
        .assign(stunden2=lambda x: x["Nextcloudaccount Vater"].map(hours_dict).fillna(0))
        .assign(stunden_summe=lambda x: x["stunden1"] + x["stunden2"])
        .assign(n_children=lambda x: x["Familie"].map(children_count))
        .assign(
            target_hours=lambda x: x.apply(
                lambda row: target_hours_dict[
                    (row["alleinerziehend"], row["n_children"])
                ],
                axis=1,
            )
        )
        .assign(progress=lambda x: np.round(np.minimum(100,x["stunden_summe"] / x["target_hours"] * 100)))
        .astype({"progress": int})
        .sort_values(by="progress", ascending=False)
        .drop(columns=["alleinerziehend", "n_children", "Nextcloudaccount Mutter", "Nextcloudaccount Vater"])
        .rename(
            columns={
                "target_hours": "Stunden SOLL",
                "stunden_summe": "Stunden IST",
                "progress": "Fortschritt",
                "stunden1": "Stunden Mutter",
                "stunden2": "Stunden Vater",
                # "n_children": "Anzahl Kinder",
            },
            errors="ignore",
        )
    )
family_hours

,Familie,Stunden Mutter,Stunden Vater,Stunden IST,Stunden SOLL,Fortschritt
0,Baldas,9.00,56.00,65.00,132,49
32,Hirsch + Lerche,31.00,14.00,45.00,102,44
6,Bessai,43.50,2.00,45.50,132,34
25,Hofmann,17.00,0.00,17.00,50,34
35,Mall + Zorn,25.00,8.75,33.75,102,33
27,Nesterova + Hoyer,17.50,25.00,42.50,132,32
11,Hadem,22.00,20.00,42.00,132,32
5,Kumbartzki + Fleig,19.50,13.00,32.50,102,32
14,Kiemle + Küster,34.00,8.00,42.00,132,32
34,Adams,10.00,18.00,28.00,102,27


In [18]:
family_hours = (
        df_names[
            [
                "Familie",
                "alleinerziehend",
                "Nextcloudaccount Mutter",
                "Nextcloudaccount Vater",
            ]
        ]
        .melt(id_vars=["Familie", "alleinerziehend"], value_name="nextcloud_account")
        .drop(columns="variable")
        .assign(n_children=lambda x: x["Familie"].map(children_count))
        .assign(
            target_hours=lambda x: x.apply(
                lambda row: target_hours_dict[
                    (row["alleinerziehend"], row["n_children"])
                ],
                axis=1,
            )
        )
        .assign(actual_hours=lambda x: x["nextcloud_account"].map(hours_dict).fillna(0))
        .groupby(["Familie", "alleinerziehend", "target_hours", "n_children"])
        .sum(numeric_only=True)
        .reset_index()
        .drop(columns=["alleinerziehend", "n_children"])
        .assign(progress=lambda x: x["actual_hours"] / x["target_hours"] * 100)
        .astype({"progress": int})
        .sort_values(by="progress", ascending=False)
        .reset_index(drop=True)
        .rename(
            columns={
                "target_hours": "Stunden SOLL",
                "actual_hours": "Stunden IST",
                "progress": "Fortschritt",
                # "n_children": "Anzahl Kinder",
            },
            errors="ignore",
        )
        .assign(Fortschritt=lambda x: np.minimum(x["Fortschritt"], 100))
    )
family_hours

,Familie,Stunden SOLL,Stunden IST,Fortschritt
0,Baldas,132,195.00,100
1,Bessai,132,91.00,68
2,Nesterova + Hoyer,132,85.00,64
3,Kiemle + Küster,132,84.00,63
4,Hadem,132,84.00,63
5,Maercks,132,65.20,49
6,Hirsch + Lerche,102,45.00,44
7,Hofmann,50,17.00,34
8,Mall + Zorn,102,33.75,33
9,Kumbartzki + Fleig,102,32.50,31


In [ ]:
from datetime import datetime
start_date = datetime(2025, 9, 1)
today = datetime.now()
elapsed_days = (today - start_date).days
percent_elapsed = np.round(elapsed_days / 365 * 100,0)
percent_elapsed

In [ ]:
fig = plt.figure(figsize=(8, 8))
plot_data = family_hours.sort_values(by="progress", ascending=True)
plot_data["okay"] = "no"
plot_data.loc[plot_data["progress"] >= percent_elapsed, "okay"] = "yes"
plot_data.loc[plot_data["progress"] >= 100, "okay"] = "done"
colors = {"yes": "black", "no": "darkred", "done": "green"}
plt.barh(
    plot_data["Familie"],
    plot_data["progress"],
    color=plot_data["okay"].map(colors),
)
plt.axvline(100, color="green", linestyle="-")
plt.axvline(percent_elapsed, color="blue", linestyle="-")
plt.text(percent_elapsed + 2, 0, f"Fortschritt\nKita-Jahr: {percent_elapsed:.0f}%", 
         verticalalignment='top', color='blue', fontsize=10)
plt.grid(axis="x", alpha=0.2)
plt.xlabel("Stunden in %")
plt.title("Kita Stundenliste")
plt.tight_layout()
plt.show()

In [ ]:
category_hours = (
    df_hours.groupby("Kategorie")
    .sum(numeric_only=True)["Stunden"]
    .sort_values(ascending=False)
    .to_frame()
)
category_hours

In [ ]:
plt.pie(category_hours["Stunden"], labels=category_hours.index, autopct="%1.1f%%")
plt.show()

In [ ]:
(
    family_hours.rename(
        columns={
            "target_hours": "Stunden SOLL",
            "actual_hours": "Stunden IST",
            "progress": "Fortschritt",
            "n_children": "Anzahl Kinder",
        }
    )
    .assign(Fortschritt=lambda x: np.minimum(x["Fortschritt"], 100))
    .drop(columns="alleinerziehend")
    .to_csv("family_hours_report.csv", index=False)
)